# Workshop about controllers

## Controller
* [Controller overview](http://www.sardana-controls.org/en/latest/devel/overview/overview_controller.html#sardana-controller-overview)
* [Writing controllers](http://www.sardana-controls.org/en/latest/devel/howto_controllers/index.html)
* [Controller API reference](http://www.sardana-controls.org/en/latest/devel/api/api_controller.html#sardana-controller-api)

### Controller overview

* Maps the communication between a pool element e.g. motor and the underlying hardware (example: a motor controller crate)
* Element axis refers to the ID of a specific hardware object (like a motor) with respect to its controller

### Writing controllers

* Before writing a new controller check in the [third-party repository](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/) if someone already wrote a similar one

* If not it does not harm to ask on one of the communication channels e.g. mailing list, github issue, etc.

### Writing controllers - common aspects

* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured

* AddDevice and DeleteDevice
 * Called on: element creation/deletion, pool start/stop and controller's code reload

### Writing controllers - common aspects

* Get axis state (State sequence)
 * Applies only to the to physical elements
 * Called on: state request, during operations e.g. motion, acquisition
 * Returns: state and status (in case of motor also returns limit switches)
 * If an exception is raised when reading the state, the axis automatically gets into the Fault state and the status contains the exception details.

* Axis extra attributes - attributes that are not included in the standard inteface e.g. close loop for a motor
* Controller extra attributes - attributes that are not included in the standard interface e.g. 
* Controller properties - similar to attributes but are foreseen for more static characteristics e.g. communication host and port

### Writing controllers - common aspects

* Controller has its own logger - `_log` member and its level can be controller with an attribute of the controller

* When developing controllers, it is very useful to call `ReloadControllerClass` and `ReloadControllerLib` commands of the pool - see [#53](https://github.com/sardana-org/sardana/issues/53)

### Synchronized start
```
/FOR/ Each controller(s) implied in the motion
     - Call PreStartAll()
/END FOR/
/FOR/ Each motor(s) implied in the motion
     - ret = PreStartOne(motor to move, new position)
     - /IF/ ret is not true
        /RAISE/ Cannot start. Motor PreStartOne returns False
     - /END IF/
     - Call StartOne(motor to move, new position)
/END FOR/
/FOR/ Each controller(s) implied in the motion
     - Call StartAll()
/END FOR/
```

### Synchronized start

[Single motor start sequence diagram](http://www.sardana-controls.org/en/latest/devel/api/api_motor.html#motion)

### Optimized hardware access while reading multiple axes (also state)

```
/FOR/ Each controller(s) implied in the reading
     - Call PreReadAll()
/END FOR/
/FOR/ Each motor(s) implied in the reading
     - PreReadOne(motor to read)
/END FOR/
/FOR/ Each controller(s) implied in the reading
     - Call ReadAll()
/END FOR/
/FOR/ Each motor(s) implied in the reading
     - Call ReadOne(motor to read)
/END FOR/
```

### Optimized hardware access while reading multiple axes (also state)

[Single motor read sequence diagram](http://www.sardana-controls.org/en/latest/devel/api/api_motor.html#motor-position)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Get motor state - `StateOne`:
 * return status of the limit switches (home, upper, lower)
 * should become Alarm if any of the overtravel limit switches becomes active - see discussion in [#507](https://github.com/sardana-org/sardana/issues/507)
* Get motor position - `ReadOne`
 * return dial position (dial position = motor position / steps per unit)
* Move a motor - `StartOne`:
 * Accepts argument: dial position (motor position = dial position * steps per unit)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Stop a motor - `StopOne`:
 * Gracefully stop a motor (deceleration and base rate should be respected)
 * Stopping multiple axis with one command should be allowed - see [#157](https://github.com/sardana-org/sardana/issues/157)
* Abort a motor - `AbortOne`:
 * Gracefully stop a motor (decelration and base rate should be respected)
 * Aborting multiple axis with one command should be allowed - see [#157](https://github.com/sardana-org/sardana/issues/157)
* Standard axis attributes - `SetAxisPar` and `GetAxisPar`
  * acceleration, deceleration, velocity, base rate and steps per unit
  * some combinations of parameters may not be coherent what to do then - see [#30](https://github.com/sardana-org/sardana/issues/30) and [#420](https://github.com/sardana-org/sardana/pull/420)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Define motor position - `DefinePosition`:
 * Loads the new motor position to the hardware
* Advanced topics:
 * Timestamp read position - return `SardanaValue` object
* Examples: IcePAP, Pmac, Tango attribute

### Example - XYZ Stage

### [How to write pseudo motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_pseudomotorcontroller.html)

* Define class members `pseudo_motor_roles` and `motor_roles`
 * Some use cases (advanced), for example HKL, may require dynamic roles - see [#86](https://github.com/sardana-org/sardana/issues/86)
 * If controller comprises only one pseduo motor, the `pseudo_motor_roles` can be omitted
* Calculate pseudo motor position - `CalcPseudo`
 * Accepts: axis, physical motor positions and the current pseudo motor positions as arguments
 * Returns calculated pseudo motor position
 * The current pseudo motor positions that arrives to `CalcPseudo` are set values and not read values TODO

### [How to write pseudo motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_pseudomotorcontroller.html)
* Calculate physical motor position - `CalcPhysical`
 * Accepts: pseudo motor positions and the current physical motor positions as arguments
 * Returns calculated physical motor position
* If great performance gain can be achived it is possible to use `CalcAllPseudo` and `CalcAllPhysical` methods
 * The default implementation iterates over axes and call the `CalcPseudo` and `CalcPhysical` methods multiple times
* Examples: Mirrors, ID, Tables, Energy, Twin motors

### Example: Mirror Table

`defctrl MirrorVerticalPseudoMotorController mvert mzc=mzc mzl=mzl mzr=mzr z=mz pitch=mpitch roll=mroll dim_x 261 dim_y 1262.5`

### CT controller

* [How to](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html)

* Uses at ALBA:
 * Electromiter
 * AdLinks
 * NI
 * Lima, Lima ROI

#### Example:

* [Network traffic](https://sourceforge.net/p/sardana/wiki/Howto-CreateControllers/attachment/myctctrl.py)


### Pseudo CT controller

* [How to](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_pseudocountercontroller.html)

* Uses at ALBA:
 * IK
 * IoverI0

### Packaging installation at ALBA

The controllers are defined at three levels:

* Production: installed via packages
* Development: getting it from repositories check out
* User: created by the users (in a shared folder)

All of them are defined in the PoolPath property.

#### Now (opensuse)
``` 
/beamlines/bl22/controls/user_ctrls  # Users
/beamlines/bl22/controls/devel/poolcontrollers # Development
/homelocal/sicilia/lib/python/site-packages/poolcontrollers # Package
/homelocal/sicilia/lib/python/site-packages/poolcontrollers/IcePAPCtrl
/homelocal/sicilia/lib/python/site-packages/poolcontrollers/IBACtrl
```
#### Future (debian)
```
/beamlines/bl22/controls/user_ctrls # Users
/beamlines/bl22/controls/devel/poolcontrollers # Development
/usr/lib/sardana/poolcontrollers # Package
/usr/lib/sardana/poolcontrollers/IcePAPCtrl
/usr/lib/sardana/poolcontrollers/IBACtrl
```


## Good/bad practices



### Use of taurus vs. use of PyTango

#### Pros Taurus

* More friendly API
* To take care with the pythonic read/write

#### Pros PyTango

* Light weigh
* Access to full Tango API (e.g. Device states)
* Allow pythonic read/write (e.g. device[attribute] = value)

TODO


### Using external libraries

The pool allows to use external libraries (not defined in the `$PYTHONPATH`) via its `PYTHONPATH` 
property. This libraries  will have more preference that the system ones. 
 

### Class inheritance

The controller inheritance is allowed in Sardana, with some limitations. If all clasess are loaded from the same path (as controllers) the sardana controller manager can fail loading the subclasess due to precedence troubles.

#### Examples:
* [TurboPmacController](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/motor/PmacCtrl/TurboPmacCtrl.py)(Controller)
 * [LtpTurboPmacController](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/motor/PmacCtrl/AlbaLtpTurboPmacCtrl.py)(Controller)

* [Ni660XCtrl](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XCTCtrl.py)(Base class) 
 * [Ni660XPositionCTCtrl](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XPositionCTCtrl.py)(Controller)
 *[Ni660XCounterCTCtrl](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XCounterCTCtrl.py)(Controller)
 * [Ni660XPulseWidthCTCtrl.py ](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XPulseWidthCTCtrl.py)(Controller)